In [1]:
import os
import json
import gzip
import pandas as pd
from urllib.request import urlopen

import random
import numpy as np
from tqdm import tqdm_notebook as tqdm
#from tqdm import tqdm.notebook.tqdm as tqdm
from collections import defaultdict

import warnings
warnings.filterwarnings("ignore")

## Se importan los dataframes 

In [2]:
# load all metadata
#hau nire diskan daukat SAD karpetan 2020-2021

video_games = []#
insurances = []   
#dfMergedfMeta= pd.read_csv('Cell_Phones/SamsungAppleXiaomiReviews_prueba.csv')
video_games= pd.read_csv('VG-Reviews5AndMetaElecNintSonyMic_v2.csv')
insurances = pd.read_csv('HRBlockIntuitReviewsTrainDev_vLast7.csv', low_memory=False)


In [3]:
video_games.keys()

Index(['Unnamed: 0', 'asin', 'vote', 'style', 'feature', 'similar_item',
       'brand', 'also_buy', 'category', 'title', 'main_cat', 'description',
       'reviewTime', 'overall', 'reviewerID', 'reviewerName', 'unixReviewTime',
       'reviewText', 'summary', 'City', 'State', 'Country', 'Postal Code'],
      dtype='object')

In [4]:
video_games['brand'].value_counts()

by\n    \n    Electronic Arts                        19949
by\n    \n    Nintendo                               16812
by\n    \n    Sony                                   10915
by\n    \n    Microsoft                               8810
Nintendo                                              4660
Electronic Arts                                       1780
Sony                                                  1697
Microsoft                                             1230
Sony Computer Entertainment                            438
by\n    \n    Sony Computer Entertainment America      106
Nintendo of America                                     98
Electronic Arts, Inc.                                   78
by\n    \n    Sony Computer Entertainment               74
Sony Entertainment                                      52
Microsoft Corporation                                   44
Super Nintendo Super Castlevania IV                     22
Electronc Arts                                          

In [5]:
insurances.keys()

Index(['Unnamed: 0', 'asin', 'brand', 'price', 'title', 'Unnamed: 5',
       'category', 'rank', 'description', 'main_cat', 'details', 'overall',
       'vote_y', 'verified_y', 'reviewTime_y', 'reviewerID_y',
       'reviewerName_y', 'reviewText', 'summary', 'unixReviewTime', 'State',
       'Country', 'Postal Code'],
      dtype='object')

In [6]:
insurances['brand'].value_counts()

Intuit                              72752
H&R Block                           16242
by\n    \n    Intuit                 9488
HRBB9                                 657
H&amp;R Block                         358
by\n    \n    H&R Block               195
Intuit, Inc.                           27
H&R BLCOK                              17
John Truby Blockbuster                 15
Video Blocks                            4
Administaff HRTools                     4
H & R Block                             4
Intuit Inc.                             3
Teneron/Block Financial Software        3
H&R                                     3
H&R BLOCK                               2
Intuit Inc./BlueHippo                   2
Name: brand, dtype: int64

## Se obtienen en dataframes aparte las distintas marcas

In [7]:
dfNintendo = video_games[video_games['brand'].str.contains('Nintendo',na=False)]

In [8]:
dfSony = video_games[video_games['brand'].str.contains('Sony|PlayStation|Electr',na=False)]

In [9]:
dfMicrosoft = video_games[video_games['brand'].str.contains('[M|m]icrosoft',na=False)]

In [10]:
dfIntuit = insurances[insurances['brand'].str.contains('Intuit')]

In [11]:
dfHR = insurances[~insurances['brand'].str.contains('Intuit')]

## Se separan las reviews negativas y positivas de cada dataframe

In [12]:
dfNintendoPositives=dfNintendo[dfNintendo['overall']>3]
dfNintendoNegatives=dfNintendo[dfNintendo['overall']<=3]

In [13]:
dfSonyPositives=dfSony[dfSony['overall']>3]
dfSonyNegatives=dfSony[dfSony['overall']<=3]

In [14]:
dfMicrosoftPositives=dfMicrosoft[dfMicrosoft['overall']>3]
dfMicrosoftNegatives=dfMicrosoft[dfMicrosoft['overall']<=3]

In [15]:
dfIntuitPositives = dfIntuit[dfIntuit['overall']>3]
dfIntuitNegatives = dfIntuit[dfIntuit['overall']<=3]

In [16]:
dfHRPositives = dfHR[dfHR['overall']>3]
dfHRNegatives = dfHR[dfHR['overall']<=3]

## Se obtienen las text reviews

In [17]:
nintendoPositivesDocuments = dfNintendoPositives[dfNintendoPositives['reviewText'].notna()]['reviewText'].tolist()
nintendoNegativesDocuments = dfNintendoNegatives[dfNintendoNegatives['reviewText'].notna()]['reviewText'].tolist()

In [18]:
sonyPositivesDocuments = dfSonyPositives[dfSonyPositives['reviewText'].notna()]['reviewText'].tolist()
sonyNegativesDocuments = dfSonyNegatives[dfSonyNegatives['reviewText'].notna()]['reviewText'].tolist()

In [19]:
microsoftPositivesDocuments = dfMicrosoftPositives[dfMicrosoftPositives['reviewText'].notna()]['reviewText'].tolist()
microsoftNegativesDocuments = dfMicrosoftNegatives[dfMicrosoftNegatives['reviewText'].notna()]['reviewText'].tolist()

In [20]:
intuitPositivesDocuments = dfIntuitPositives[dfIntuitPositives['reviewText'].notna()]['reviewText'].tolist()
intuitNegativesDocuments = dfIntuitNegatives[dfIntuitNegatives['reviewText'].notna()]['reviewText'].tolist()

In [21]:
HRPositivesDocuments = dfHRPositives[dfHRPositives['reviewText'].notna()]['reviewText'].tolist()
HRNegativesDocuments = dfHRNegatives[dfHRNegatives['reviewText'].notna()]['reviewText'].tolist()

## LDA

In [22]:
def display_topics(H, W, feature_names, documents, test, no_top_words = 30, no_top_documents = 10):
    f = open(test + ".txt", "a")
    for topic_idx, topic in enumerate(H):
        
        topic = "Topic %d:" % (topic_idx)
        f.write(topic + '\n\n')
        print(topic)
        
        feature_names = " ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]])
        f.write(feature_names + '\n\n')
        print(feature_names)
        
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for doc_index in top_doc_indices:
            print(documents[doc_index])
            f.write(documents[doc_index] + '\n')
    f.close()

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

### LDASonyNeg_29_1.9

In [24]:
no_topics = 29

alpha = 1.9
beta = 1.9

documents = sonyNegativesDocuments

In [25]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDASonyNeg_29_1.9')

### LDAMicroPos_21_1.9

In [ ]:
no_topics = 21

alpha = 1.9
beta = 1.9

documents = microsoftPositivesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAMicroPos_21_1.9')

### LDAMicroNeg_16_1.9

In [ ]:
no_topics = 16

alpha = 1.9
beta = 1.9

documents = microsoftNegativesDocuments

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(documents)
tf_feature_names = tf_vectorizer.get_feature_names()

In [ ]:
lda_model = LatentDirichletAllocation(n_components=no_topics, doc_topic_prior=alpha, topic_word_prior=beta, max_iter=100, learning_method='online', learning_offset=50.,random_state=0).fit(tf)
lda_W = lda_model.transform(tf)
lda_H = lda_model.components_

In [ ]:
display_topics(lda_H, lda_W, tf_feature_names, documents, 'LDAMicroNeg_16_1.9')

### GensimMicroPos

### GensimMicroNeg